---
# Analisis Putusan Perdata Khusus Merek Menggunakan Case-Based Reasoning Berbasis TF-IDF: Studi Perbandingan SVM dan Naive Bayes

Anggota Kelompok :

1. Adam Kurniawan - 202210370311211
2. Wafiq Azizah - 202210370311247

## Tugas Besar Mata Kuliah Penalaran Komputer (B)
---



## Tahap 1 – Membangun Case Base

### Tujuan
Tahap ini bertujuan untuk mengumpulkan dan membersihkan dokumen putusan pengadilan perdata khusus merek untuk membentuk basis kasus yang terstruktur sebagai fondasi proses reasoning dalam sistem Case-Based Reasoning.

---

### Langkah Kerja

#### 1. Seleksi dan Pengunduhan Dokumen

- Jenis perkara yang dipilih adalah **Perdata Khusus terkait Merek**.
- Data diambil melalui proses scraping dari situs resmi putusan Mahkamah Agung.
- Dokumen hasil unduhan disimpan dalam format PDF pada folder `Merek_PDF/`.

---

#### 2. Konversi dan Ekstraksi Teks

- File PDF diubah menjadi format teks dengan bantuan library `pdfminer`.
- Setiap dokumen diproses untuk mengekstrak keseluruhan isi putusan dalam bentuk teks mentah.
- Hasil ekstraksi disimpan secara terpisah untuk setiap dokumen di direktori `data/raw/`.

---

#### 3. Pembersihan Teks (Cleaning)

- Elemen seperti watermark, header, footer, dan disclaimer resmi dari Mahkamah Agung dihapus dari dokumen.
- Proses normalisasi teks mencakup:
  - Penghapusan baris kosong yang berulang.
  - Konversi seluruh huruf menjadi huruf kecil.
  - Penghilangan spasi yang tidak perlu.
- Rasio retensi konten digunakan untuk menilai sejauh mana informasi utama tetap terjaga setelah proses pembersihan.

---

#### 4. Validasi dan Logging

- Setiap file yang telah diproses dicatat dalam log dengan informasi berikut:  
  - Nomor dokumen
  - Status hasil pembersihan (OK atau rendah)
  - Direktori penyimpanan
  - Waktu dan tanggal pemrosesan
- Seluruh catatan log disimpan dalam file `logs/cleaning.log`.

---

### Output Tahap Ini

- Jumlah total dokumen yang berhasil diproses sebanyak **70 kasus**.
- Teks hasil pembersihan disimpan di direktori `data/raw/`.
- Informasi proses dan status pembersihan tercatat dalam file log `logs/cleaning.log`.

---

Tahap awal ini berhasil menghasilkan korpus dasar untuk mendukung sistem CBR. Lebih dari 80% dokumen telah berhasil dikonversi ke teks dan dibersihkan dengan metode terstruktur, sehingga siap digunakan pada proses representasi berikutnya.





In [4]:
import os
import re
import glob
from pdfminer.high_level import extract_text
from datetime import datetime

# === Directory configuration ===
SOURCE_DIR = 'Merek_PDF'
DEST_DIR = 'data/raw'
LOG_DEST = 'logs/cleaning.log'

def purify_text(input_text):
    """Remove unwanted headers, footers, and disclaimers from extracted text."""
    initial_length = len(input_text) if input_text else 1  # Prevent division by zero

    # Eliminate watermarks and page indicators
    processed_text = input_text
    processed_text = processed_text.replace("Direktori Putusan Mahkamah Agung Republik Indonesia", "")
    processed_text = processed_text.replace("putusan.mahkamahagung.go.id", "")
    processed_text = re.sub(r'halaman\s*\d+', '', processed_text, flags=re.IGNORECASE)
    processed_text = processed_text.replace("M a h ka m a h A g u n g R e p u blik In d o n esia\n", "")
    
    # Record length after initial cleaning
    intermediate_length = len(processed_text)
    
    # Remove disclaimer content
    processed_text = processed_text.replace("Disclaimer\n", "")
    processed_text = processed_text.replace(
        "Kepaniteraan Mahkamah Agung Republik Indonesia berusaha untuk selalu mencantumkan informasi paling kini dan akurat sebagai bentuk komitmen Mahkamah Agung untuk pelayanan publik, transparansi dan akuntabilitas\n", "")
    processed_text = processed_text.replace(
        "pelaksanaan fungsi peradilan. Namun dalam hal-hal tertentu masih dimungkinkan terjadi permasalahan teknis terkait dengan akurasi dan keterkinian informasi yang kami sajikan, hal mana akan terus kami perbaiki dari waktu kewaktu.\n", "")
    processed_text = processed_text.replace(
        "Dalam hal Anda menemukan inakurasi informasi yang termuat pada situs ini atau informasi yang seharusnya ada, namun belum tersedia, maka harap segera hubungi Kepaniteraan Mahkamah Agung RI melalui :\n", "")
    processed_text = processed_text.replace(
        "Email : kepaniteraan@mahkamahagung.go.id    Telp : 021-384 3348 (ext.318)\n", "")

    # Final text normalization
    processed_text = processed_text.lower()
    processed_text = ' '.join(processed_text.split())

    # Calculate content retention ratio
    retention_ratio = intermediate_length / initial_length

    return processed_text, retention_ratio

def record_log(case_id, retention_ratio):
    """Log processing details to a file."""
    os.makedirs(os.path.dirname(LOG_DEST), exist_ok=True)
    with open(LOG_DEST, 'a', encoding='utf-8') as log:
        timestamp = datetime.now().strftime("%Y-%m-%dT%H:%M:%S")
        log.write(f"[{timestamp}] {case_id} | Content Integrity: {retention_ratio:.2%}\n")

def handle_pdf_batch():
    """Process all PDFs in the source directory."""
    os.makedirs(DEST_DIR, exist_ok=True)
    pdf_paths = sorted(glob.glob(os.path.join(SOURCE_DIR, '*.pdf')))
    
    if not pdf_paths:
        print("No PDFs found in the source directory.")
        return

    for index, pdf in enumerate(pdf_paths, 1):
        case_id = f"case_{index:03d}"
        output_path = os.path.join(DEST_DIR, f"{case_id}.txt")

        try:
            # Extract text from PDF
            raw_text = extract_text(pdf)

            # Clean and validate text
            final_text, integrity_ratio = purify_text(raw_text)

            # Check content integrity
            if integrity_ratio < 0.8:
                print(f"[WARNING] {case_id} retains only {integrity_ratio:.2%} of content.")
            else:
                print(f"[OK] {case_id} processed ({integrity_ratio:.2%} valid).")
                # Save cleaned text if valid
                with open(output_path, 'w', encoding='utf-8') as output:
                    output.write(final_text)

            # Log the result
            record_log(case_id, integrity_ratio)

        except Exception as err:
            print(f"[ERROR] Failed to process {pdf}: {str(err)}")

def main():
    """Initiate PDF processing workflow."""
    print("Beginning PDF text extraction...")
    handle_pdf_batch()
    print("PDF processing completed.")

if __name__ == "__main__":
    main()

Beginning PDF text extraction...
[OK] case_001 processed (96.43% valid).
[OK] case_002 processed (96.25% valid).
[OK] case_003 processed (96.56% valid).
[OK] case_004 processed (96.38% valid).
[OK] case_005 processed (96.34% valid).
[OK] case_006 processed (96.41% valid).
[OK] case_007 processed (96.38% valid).
[OK] case_008 processed (96.22% valid).
[OK] case_009 processed (96.46% valid).
[OK] case_010 processed (96.53% valid).
[OK] case_011 processed (96.10% valid).
[OK] case_012 processed (96.40% valid).
[OK] case_013 processed (96.59% valid).
[OK] case_014 processed (96.26% valid).
[OK] case_015 processed (96.03% valid).
[OK] case_016 processed (96.59% valid).
[OK] case_017 processed (96.82% valid).
[OK] case_018 processed (96.26% valid).
[OK] case_019 processed (96.38% valid).
[OK] case_020 processed (96.53% valid).
[OK] case_021 processed (96.29% valid).
[OK] case_022 processed (96.28% valid).
[OK] case_023 processed (96.16% valid).
[OK] case_024 processed (96.64% valid).
[OK] ca

## Tahap 2 – Case Representation

### Tujuan

Mewakili setiap dokumen putusan dalam bentuk vektor fitur menggunakan pendekatan TF-IDF agar dapat digunakan sebagai input dalam sistem Case-Based Reasoning, serta mempersiapkan data untuk proses klasifikasi menggunakan algoritma SVM dan Naive Bayes.

---

### Langkah Kerja

#### 1. Ekstraksi Metadata

Setiap dokumen hasil pembersihan dianalisis untuk mengambil informasi penting sebagai metadata menggunakan metode berbasis pola (regex) dan teknik pemrosesan teks. Metadata yang diambil mencakup:

- Nomor perkara (`no_perkara`)
- Tanggal putusan (`tanggal`)
- Ringkasan fakta hukum (`ringkasan_fakta`)
- Dasar hukum atau pasal yang digunakan (`pasal`)
- Identitas pihak-pihak yang bersengketa (penggugat dan tergugat)
- Konten lengkap putusan (`text_full`)

---

#### 2. Penyimpanan Data Terstruktur

Output metadata yang telah diekstraksi disimpan dalam dua format file guna mempermudah tahap pemrosesan selanjutnya:

- **CSV** disimpan di: `data/processed/cases_extracted.csv`
- **JSON** disimpan di: `data/processed/cases_extracted.json`

Setiap entri kasus memiliki struktur data yang terdiri dari beberapa atribut, yaitu:

- `case_id`
- `no_perkara`
- `tanggal`
- `ringkasan_fakta`
- `pasal`
- `pihak`
- `text_full`

---

#### 3. Feature Engineering

Untuk memperkaya kualitas representasi setiap kasus, dilakukan tahapan rekayasa fitur (feature engineering) dengan beberapa pendekatan berikut:

- **Panjang Teks**: Mengukur jumlah kata yang terdapat dalam kolom `ringkasan_fakta`.
- **Bag-of-Words (BoW)**: Mengidentifikasi dan menghitung frekuensi kemunculan kata dalam ringkasan untuk membentuk representasi fitur.
- **Pasangan QA Sederhana**: Menyusun pertanyaan dan jawaban otomatis berdasarkan informasi dalam ringkasan, contohnya:

  - Berapa nomor perkara dalam kasus ini?
  - Dasar hukum atau pasal apa yang diterapkan?
  - Siapa yang menjadi tergugat dalam perkara tersebut?
  - Bagaimana ringkasan fakta dari kasus ini?

---

#### 4. Penyimpanan Fitur

Fitur-fitur hasil rekayasa disimpan dalam format JSON untuk memudahkan penggunaan dalam tahap analisis berikutnya, dengan rincian sebagai berikut:

- `data/processed/features_length.json` – berisi informasi panjang teks
- `data/processed/features_bow.json` – mencakup representasi frekuensi kata
- `data/processed/features_qa_pairs.json` – menyimpan pasangan pertanyaan dan jawaban dari ringkasan

---

Tahap representasi berhasil menyusun data setiap kasus dalam format yang terstruktur dan sistematis. Seluruh kasus kini dilengkapi dengan metadata utama serta fitur tambahan yang dirancang untuk memperkuat proses penelusuran dan penalaran pada tahap selanjutnya dalam sistem Case-Based Reasoning.

In [5]:
import os
import re
import json
import csv
from collections import Counter

RAW_FOLDER = 'data/raw'
PROCESSED_FOLDER = 'data/processed'

os.makedirs(PROCESSED_FOLDER, exist_ok=True)

def extract_metadata(text):
    metadata = {}

    # Nomor Perkara
    match = re.search(r'Nomor\s*:\s*(\S+)', text, re.IGNORECASE)
    metadata['no_perkara'] = match.group(1) if match else ''

    # Tanggal Putusan
    match = re.search(r'\b(?:putusan|diputuskan)\s*(?:pada|tanggal)?\s*(\d{1,2}\s+\w+\s+\d{4})', text, re.IGNORECASE)
    metadata['tanggal'] = match.group(1) if match else ''

    # Pasal yang disebutkan
    match = re.findall(r'Pasal\s+\d+\s+[^\n.,;]*', text, re.IGNORECASE)
    metadata['pasal'] = match if match else []

    # Ringkasan Fakta Sederhana
    match = re.search(r'(menimbang\s+bahwa.*?)((?:menimbang|mengingat|memperhatikan)\b.*)', text, re.IGNORECASE | re.DOTALL)
    metadata['ringkasan_fakta'] = match.group(1).strip() if match else ''

    # Terdakwa & Korban (opsional)
    terdakwa = re.findall(r'terdakwa(?:\s*:\s*|\s+)([A-Z][a-zA-Z\s]+)', text)
    korban = re.findall(r'korban(?:\s*:\s*|\s+)([A-Z][a-zA-Z\s]+)', text)
    metadata['pihak'] = {
        'terdakwa': terdakwa[0] if terdakwa else '',
        'korban': korban[0] if korban else ''
    }

    return metadata

def feature_engineering(text, metadata):
    tokens = re.findall(r'\b\w+\b', text.lower())
    length = len(tokens)
    bow = dict(Counter(tokens))

    qa_pairs = {
        "Berapa nomor perkara dalam kasus ini?": metadata.get("no_perkara", ""),
        "Dasar hukum atau pasal apa yang diterapkan?": ", ".join(metadata.get("pasal", [])),
        "Siapa yang menjadi tergugat dalam perkara tersebut?": metadata.get("pihak", {}).get("terdakwa", ""),
        "Bagaimana ringkasan fakta dari kasus ini?": metadata.get("pihak", {}).get("korban", "")
    }

    return length, bow, qa_pairs

def process_all():
    # === Cek folder dan file .txt ===
    if not os.path.exists(RAW_FOLDER):
        raise FileNotFoundError(f"[ERROR] Folder '{RAW_FOLDER}' tidak ditemukan. Harap buat dan isi dengan file .txt.")

    txt_files = sorted([f for f in os.listdir(RAW_FOLDER) if f.endswith('.txt')])
    if not txt_files:
        raise FileNotFoundError(f"[ERROR] Tidak ada file .txt dalam '{RAW_FOLDER}'. Harap isi folder dengan putusan .txt.")

    all_cases = []
    features_length = {}
    features_bow = {}
    features_qa = {}

    for i, fname in enumerate(txt_files, 1):
        path = os.path.join(RAW_FOLDER, fname)
        with open(path, 'r', encoding='utf-8') as file:
            text = file.read().strip()

        if not text:
            print(f"[PERINGATAN] File kosong dilewati: {fname}")
            continue

        case_id = f"case_{i:03d}"
        metadata = extract_metadata(text)
        metadata['case_id'] = case_id
        metadata['text_full'] = text

        length, bow, qa = feature_engineering(text, metadata)

        all_cases.append(metadata)
        features_length[case_id] = length
        features_bow[case_id] = bow
        features_qa[case_id] = qa

    # Simpan CSV
    csv_path = os.path.join(PROCESSED_FOLDER, 'cases_extracted.csv')
    with open(csv_path, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['case_id', 'no_perkara', 'tanggal', 'ringkasan_fakta', 'pasal', 'pihak', 'text_full']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for row in all_cases:
            writer.writerow(row)

    # Simpan JSON
    json_path = os.path.join(PROCESSED_FOLDER, 'cases_extracted.json')
    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(all_cases, f, ensure_ascii=False, indent=2)

    # Simpan fitur
    with open(os.path.join(PROCESSED_FOLDER, 'features_length.json'), 'w', encoding='utf-8') as f:
        json.dump(features_length, f, ensure_ascii=False, indent=2)
    with open(os.path.join(PROCESSED_FOLDER, 'features_bow.json'), 'w', encoding='utf-8') as f:
        json.dump(features_bow, f, ensure_ascii=False, indent=2)
    with open(os.path.join(PROCESSED_FOLDER, 'features_qa_pairs.json'), 'w', encoding='utf-8') as f:
        json.dump(features_qa, f, ensure_ascii=False, indent=2)

    print(f"[SUKSES] {len(all_cases)} kasus berhasil diproses dan disimpan:")
    print("CSV →", csv_path)
    print("JSON →", json_path)
    print("Length →", os.path.join(PROCESSED_FOLDER, 'features_length.json'))
    print("BoW →", os.path.join(PROCESSED_FOLDER, 'features_bow.json'))
    print("QA Pairs →", os.path.join(PROCESSED_FOLDER, 'features_qa_pairs.json'))

if __name__ == "__main__":
    process_all()


[SUKSES] 70 kasus berhasil diproses dan disimpan:
CSV → data/processed\cases_extracted.csv
JSON → data/processed\cases_extracted.json
Length → data/processed\features_length.json
BoW → data/processed\features_bow.json
QA Pairs → data/processed\features_qa_pairs.json


## Tahap 3 – Case Retrieval

### Tujuan

Mengembangkan mekanisme pencarian kasus serupa berdasarkan kemiripan teks menggunakan representasi TF-IDF, untuk menemukan putusan terdahulu yang relevan sebagai referensi dalam sistem Case-Based Reasoning, serta mengevaluasi kinerja pencarian dengan algoritma SVM dan Naive Bayes.

---

### Langkah Kerja

#### 1. Representasi Vektor

Setiap ringkasan fakta dari putusan diolah menjadi vektor numerik dengan pendekatan **TF-IDF (Term Frequency – Inverse Document Frequency)**. Konversi ini bertujuan untuk merepresentasikan teks dalam format angka sehingga dapat digunakan dalam perhitungan tingkat kemiripan antar dokumen dan proses klasifikasi lanjutan.

---

#### 2. Splitting Data

Dataset dibagi ke dalam dua subset, yaitu:

- **Data pelatihan (training set)** sebesar 80%
- **Data pengujian (test set)** sebesar 20%

Pembagian ini digunakan untuk melatih model klasifikasi yang menggabungkan TF-IDF dengan algoritma SVM, serta untuk melakukan evaluasi awal terhadap efektivitas proses pencarian kasus serupa.

---

#### 3. Model Retrieval

Terdapat dua metode yang digunakan dalam proses pencarian kasus serupa (retrieval):

- **a. TF-IDF + Cosine Similarity (Pendekatan CBR)**
  Mengukur kemiripan antara vektor TF-IDF dari query dan seluruh kasus dalam basis data menggunakan cosine similarity. Kasus-kasus dengan skor tertinggi (top-k) dianggap paling relevan.

- **b. TF-IDF + Support Vector Machine (SVM)**
  Menggunakan pendekatan supervised learning, di mana model SVM dilatih untuk menghubungkan ringkasan fakta dengan `case_id`. Ketika menerima sebuah query, model akan memprediksi `case_id` yang paling cocok.

---

#### 4. Fungsi Retrieval

Fungsi pencarian (retrieval) dikembangkan secara terpisah untuk masing-masing metode:

- `retrieve_cosine(query: str, k: int = 5)`
  Menghasilkan top-k `case_id` dengan nilai kemiripan cosine tertinggi terhadap query dari seluruh basis kasus.

- `retrieve_svm(query: str)`
  Mengembalikan satu `case_id` hasil prediksi model SVM berdasarkan klasifikasi ringkasan fakta dari query yang diberikan.

---

#### 5. Pengujian Awal

Sebanyak 10 query uji disusun lengkap dengan `case_id` yang menjadi ground-truth. Untuk setiap query:

- Dihitung top-5 hasil teratas berdasarkan skor cosine similarity.
- Diprediksi satu hasil teratas menggunakan model SVM.
- Kedua hasil tersebut kemudian dibandingkan dengan ground-truth untuk evaluasi akurasi.

Seluruh data query uji disimpan dalam file:

- `data/eval/queries.json`

---

### Output

Tahap ini menghasilkan beberapa file penting sebagai output, yaitu:

- Model klasifikasi SVM disimpan dalam file: `03_retrieval_model.pkl`
- Objek TF-IDF vectorizer disimpan dalam: `03_vectorizer.pkl`
- Dataset berisi query uji tersimpan di: `data/eval/queries.json`

---

Proses Case Retrieval telah selesai diterapkan dengan dua pendekatan berbeda: metode unsupervised berbasis kemiripan teks dan metode supervised melalui model klasifikasi. Keduanya telah disiapkan untuk mendukung proses evaluasi dan prediksi kasus pada tahapan selanjutnya.

In [2]:
import os
import json
import joblib
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

# === Konfigurasi path ===
CSV_PATH = "data/processed/cases_extracted.csv"
MODEL_PATH = "03_retrieval_model.pkl"
VECTORIZER_PATH = "03_vectorizer.pkl"
QUERY_PATH = "data/eval/queries.json"
os.makedirs("data/eval", exist_ok=True)

# === Muat data CSV ===
df = pd.read_csv(CSV_PATH)
texts = df['ringkasan_fakta'].fillna(df['text_full']).astype(str)
labels = df['case_id'].astype(str)

# === Split data 80:20 ===
X_train, X_test, y_train, y_test = train_test_split(
    texts, labels, test_size=0.2, random_state=42
)

# === TF-IDF + SVM ===
vectorizer = TfidfVectorizer(max_features=3000, ngram_range=(1,2))
X_train_vec = vectorizer.fit_transform(X_train)

classifier = LinearSVC()
classifier.fit(X_train_vec, y_train)

# === Simpan model dan vectorizer ===
joblib.dump(classifier, MODEL_PATH)
joblib.dump(vectorizer, VECTORIZER_PATH)

# === Ambil 10 query uji ===
sample_df = df[df['ringkasan_fakta'].notnull() & (df['ringkasan_fakta'].str.strip() != '')]
sample_df = sample_df.sample(n=10, random_state=42)

sample_queries = [
    {
        "query": row["ringkasan_fakta"][:500],
        "ground_truth": row["case_id"]
    }
    for _, row in sample_df.iterrows()
]

with open(QUERY_PATH, "w", encoding="utf-8") as f:
    json.dump(sample_queries, f, indent=2, ensure_ascii=False)

print("[SUKSES] Model disimpan di :", MODEL_PATH)
print("[SUKSES] Vectorizer disimpan di :", VECTORIZER_PATH)
print("[SUKSES] 10 query uji disimpan di :", QUERY_PATH)

[SUKSES] Model disimpan di : 03_retrieval_model.pkl
[SUKSES] Vectorizer disimpan di : 03_vectorizer.pkl
[SUKSES] 10 query uji disimpan di : data/eval/queries.json


c:\MATA KULIAH\Semester 6\Penalaran Komputer\coba\putusan_pidana-khusus\.conda\Lib\site-packages\sklearn\utils\multiclass.py:213: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  y_type = type_of_target(y, input_name="y")


### Tahap 4 – Solution Reuse

#### Tujuan

Mengambil dan memanfaatkan solusi dari kasus terdahulu yang paling relevan berdasarkan hasil retrieval, untuk memberikan rekomendasi atau prediksi putusan pada kasus baru dalam sistem Case-Based Reasoning.

---

#### Langkah Kerja

1. **Ekstraksi Solusi**
   - Untuk setiap kasus terdahulu yang berhasil ditemukan pada tahap sebelumnya, bagian **amar putusan** atau segmen akhir dari dokumen diekstrak sebagai `solusi_text`.
   - Informasi amar putusan ini disimpan dalam struktur data berupa pasangan `{case_id: solusi_text}`.

2. **Algoritma Prediksi Solusi**

Dua pendekatan utama digunakan dalam proses prediksi solusi:

- **CBR (TF-IDF + Cosine Similarity)**
  Sistem menyeleksi *top-k* kasus yang paling mirip berdasarkan nilai cosine similarity, kemudian menerapkan salah satu dari tiga strategi berikut:

  - `weighted`: memilih solusi dari kasus dengan akumulasi skor similarity tertinggi.
  - `majority`: menentukan solusi berdasarkan frekuensi kemunculan terbanyak di antara *top-k*.
  - `hybrid`: menggabungkan pertimbangan skor similarity dan frekuensi solusi.

- **Supervised (TF-IDF + SVM)**
  Sistem menggunakan model klasifikasi SVM untuk memprediksi `case_id` yang paling sesuai, lalu mengambil solusi dari kasus tersebut sebagai hasil prediksi.

3. **Ringkasan Solusi**

   - Solusi yang terpilih dipersingkat dengan mengambil 300 karakter pertama dari teks, sehingga menghasilkan ringkasan yang padat namun tetap informatif.

4. **Uji Coba Sistem**

   - Sistem dijalankan menggunakan 10 query uji yang terdapat pada file `queries.json` dari tahap sebelumnya.
   - Untuk setiap query, digunakan dua fungsi: `predict_with_cosine()` dan `predict_with_svm()` guna menguji performa kedua pendekatan prediksi solusi.

---

#### Fungsi Utama

- `predict_with_cosine(query: str, k=5, mode="weighted")`
  Melakukan pencarian terhadap *top-k* kasus paling relevan menggunakan cosine similarity, lalu menentukan solusi akhir dengan tiga opsi strategi:

  - `"weighted"`: memilih solusi dari kasus dengan akumulasi skor similarity tertinggi.
  - `"majority"`: memilih solusi yang paling sering muncul di antara hasil pencarian.
  - `"hybrid"`: menggabungkan skor similarity dan frekuensi kemunculan solusi.

- `predict_with_svm(query: str)`
  Memprediksi `case_id` menggunakan model klasifikasi SVM, kemudian mengambil solusi dari kasus yang telah dipetakan tersebut.

---

#### Output

Dua file CSV utama dihasilkan sebagai output dari proses prediksi:

- `data/results/predictions_cosine.csv`
  Berisi hasil prediksi solusi menggunakan pendekatan CBR dengan Cosine Similarity, menggunakan mode `weighted` sebagai pengaturan default.

- `data/results/predictions_svm.csv`
  Menyimpan hasil prediksi solusi berdasarkan model klasifikasi SVM.

Setiap file memiliki kolom-kolom sebagai berikut:

- `query_id`: Nomor identifikasi masing-masing query.
- `query`: Ringkasan fakta dari kasus baru.
- `predicted_solution`: Solusi yang dihasilkan berdasarkan kasus terdekat.
- `top_5_case_ids`: Daftar lima case ID teratas hasil retrieval (untuk metode Cosine).
- `top_1_case_id`: Case ID hasil prediksi tunggal dari model SVM.





In [3]:
import os
import json
import joblib
import pandas as pd
from collections import defaultdict, Counter
from sklearn.metrics.pairwise import cosine_similarity

# === PATH KONFIGURASI ===
CASE_DATA = "data/processed/cases_extracted.csv"
QUERY_PATH = "data/eval/queries.json"
MODEL_PATH = "03_retrieval_model.pkl"
VECTORIZER_PATH = "03_vectorizer.pkl"
COSINE_OUTPUT = "data/results/predictions_cosine.csv"
SVM_OUTPUT = "data/results/predictions_svm.csv"
os.makedirs("data/results", exist_ok=True)

# === Load data dan model ===
df = pd.read_csv(CASE_DATA)
with open(QUERY_PATH, encoding="utf-8") as f:
    queries = json.load(f)

classifier = joblib.load(MODEL_PATH)
vectorizer = joblib.load(VECTORIZER_PATH)

# === Siapkan korpus dokumen dan solusi ===
texts = df["ringkasan_fakta"].fillna(df["text_full"]).astype(str).tolist()
case_ids = df["case_id"].astype(str).tolist()
solutions = df["text_full"].astype(str).tolist()

tfidf_matrix = vectorizer.transform(texts)
case_solutions = dict(zip(case_ids, solutions))
case_id_to_text = dict(zip(case_ids, texts))

# === COSINE SIMILARITY DENGAN MODE: majority / weighted / hybrid ===
def predict_with_cosine(query: str, k: int = 5, mode: str = "weighted"):
    query_vec = vectorizer.transform([query])
    scores = cosine_similarity(query_vec, tfidf_matrix).flatten()
    top_k_idx = scores.argsort()[::-1][:k]
    
    solution_scores = defaultdict(float)
    solution_counts = Counter()
    
    for i in top_k_idx:
        cid = case_ids[i]
        score = float(scores[i])
        solution = case_solutions.get(cid, "")[:300]

        if mode == "weighted":
            solution_scores[solution] += score
        elif mode == "majority":
            solution_counts[solution] += 1
        elif mode == "hybrid":
            solution_scores[solution] += score * 0.7 + solution_counts[solution] * 0.3
        else:
            raise ValueError(f"Mode tidak dikenali: {mode}")

    # Pilih solusi terbaik
    if mode in ("weighted", "hybrid"):
        predicted_solution = max(solution_scores.items(), key=lambda x: x[1])[0]
    else:  # majority
        predicted_solution = solution_counts.most_common(1)[0][0]

    top_ids = [case_ids[i] for i in top_k_idx]
    return predicted_solution, top_ids

# === SVM CLASSIFICATION ===
def predict_with_svm(query: str):
    query_vec = vectorizer.transform([query])
    pred_case_id = classifier.predict(query_vec)[0]

    sol = case_solutions.get(str(pred_case_id), "")
    return sol[:300], [pred_case_id]

# === Jalankan untuk semua query ===
cosine_results = []
svm_results = []

for i, q in enumerate(queries, 1):
    query_text = q["query"]

    # COSINE - mode: "weighted" | "majority" | "hybrid"
    cosine_pred, cosine_ids = predict_with_cosine(query_text, mode="weighted")
    cosine_results.append({
        "query_id": i,
        "query": query_text,
        "predicted_solution": cosine_pred,
        "top_5_case_ids": cosine_ids
    })

    # SVM
    svm_pred, svm_id = predict_with_svm(query_text)
    svm_results.append({
        "query_id": i,
        "query": query_text,
        "predicted_solution": svm_pred,
        "top_1_case_id": svm_id[0]
    })

# === Simpan hasil ke CSV ===
pd.DataFrame(cosine_results).to_csv(COSINE_OUTPUT, index=False)
pd.DataFrame(svm_results).to_csv(SVM_OUTPUT, index=False)

print("[SUKSES] Hasil prediksi COSINE disimpan di:", COSINE_OUTPUT)
print("[SUKSES] Hasil prediksi SVM disimpan di   :", SVM_OUTPUT)

[SUKSES] Hasil prediksi COSINE disimpan di: data/results/predictions_cosine.csv
[SUKSES] Hasil prediksi SVM disimpan di   : data/results/predictions_svm.csv


### Tahap 5 – Model Evaluation

#### Tujuan

Mengevaluasi performa sistem Case-Based Reasoning dalam memprediksi solusi hukum dengan membandingkan hasil prediksi dari pendekatan CBR berbasis Cosine Similarity dan klasifikasi SVM terhadap data uji, menggunakan metrik evaluasi seperti akurasi, precision, dan recall untuk menilai efektivitas masing-masing metode.

---

#### Langkah Kerja

##### 1. Evaluasi Retrieval

- Evaluasi difokuskan pada hasil *top-k retrieval* menggunakan pendekatan **TF-IDF + Cosine Similarity**.
- Untuk setiap query, sistem memverifikasi apakah `ground_truth` terdapat di antara *top-k* kasus yang diambil.
- Metrik yang digunakan untuk menilai kinerja meliputi:
  - **Accuracy**
  - **Precision**
  - **Recall**
  - **F1-Score**
- Seluruh perhitungan dilakukan menggunakan fungsi-fungsi yang tersedia di modul `sklearn.metrics`.

##### 2. Evaluasi Prediksi

- Evaluasi dilakukan pada hasil prediksi tunggal (*top-1*) yang dihasilkan oleh pendekatan **TF-IDF + SVM**.
- Setiap query menghasilkan satu `case_id` yang diprediksi, kemudian dibandingkan langsung dengan `ground_truth` untuk mengukur tingkat kesesuaian.
- Metrik evaluasi yang digunakan tetap konsisten dengan evaluasi retrieval, yaitu:
  - **Accuracy**
  - **Precision**
  - **Recall**
  - **F1-Score**

##### 3. Visualisasi & Laporan

- Hasil evaluasi divisualisasikan dalam bentuk **grafik batang (bar chart)** untuk memperlihatkan perbandingan performa antar model secara jelas.
- Dilakukan pula **analisis kesalahan (error analysis)** terhadap query-query yang prediksinya tidak sesuai dengan `ground_truth`, dan hasil analisis tersebut disimpan dalam format file JSON untuk keperluan peninjauan lebih lanjut.

---

#### Implementasi

- Fungsi utama untuk evaluasi mencakup:

  - `eval_retrieval()`: mengevaluasi performa pencarian top-k menggunakan cosine similarity.
  - `eval_prediction()`: mengukur akurasi prediksi top-1 yang dihasilkan oleh model SVM.

- Fungsi pendukung:

  - `save_errors()`: mencatat dan menyimpan query yang hasil prediksinya tidak sesuai dengan ground truth.

- Evaluasi dilakukan dengan mengacu pada file hasil prediksi berikut:

  - `data/results/predictions_cosine.csv`
  - `data/results/predictions_svm.csv`

- File referensi jawaban yang benar (ground truth) terdapat pada:

  - `data/eval/queries.json`

---

#### Output
- **Output Evaluasi**

  - `data/eval/retrieval_metrics.csv`
    Berisi hasil pengukuran metrik evaluasi untuk pendekatan TF-IDF dengan Cosine Similarity.
  - `data/eval/prediction_metrics.csv`
    Berisi nilai metrik evaluasi untuk pendekatan TF-IDF dengan model SVM.

- **Grafik Performa**

  - `data/eval/performance_comparison.png`
    Visualisasi berupa grafik batang yang membandingkan performa antar model berdasarkan metrik evaluasi.

- **Analisis Kesalahan (Error Analysis)**

  - `data/eval/error_cases_cosine.json`
    Menyimpan daftar query yang tidak berhasil ditemukan secara akurat menggunakan metode cosine similarity.
  - `data/eval/error_cases_svm.json`
    Menyimpan query yang diprediksi salah oleh pendekatan berbasis SVM.

---

Tahap ini berperan penting dalam menilai sejauh mana sistem CBR efektif diterapkan pada kasus perdata khusus merek. Hasil evaluasi menjadi acuan utama dalam menentukan model terbaik yang akan digunakan untuk proses deployment maupun pengembangan sistem selanjutnya.

In [1]:
import os
import json
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from typing import Dict

# === PATH SETUP ===
COSINE_PRED = "data/results/predictions_cosine.csv"
SVM_PRED = "data/results/predictions_svm.csv"
EVAL_QUERIES = "data/eval/queries.json"
RETRIEVAL_METRIC_OUTPUT = "data/eval/retrieval_metrics.csv"
PREDICTION_METRIC_OUTPUT = "data/eval/prediction_metrics.csv"
ERROR_COSINE_OUTPUT = "data/eval/error_cases_cosine.json"
ERROR_SVM_OUTPUT = "data/eval/error_cases_svm.json"
os.makedirs("data/eval", exist_ok=True)

# === LOAD GROUND TRUTH ===
with open(EVAL_QUERIES, encoding='utf-8') as f:
    ground_truth_data = json.load(f)
    gt_dict = {i + 1: str(item["ground_truth"]).strip() for i, item in enumerate(ground_truth_data)}

# === EVALUASI RETRIEVAL COSINE (TOP-K) ===
def eval_retrieval(pred_file: str, model_name: str, k: int = 5) -> Dict:
    df = pd.read_csv(pred_file)
    y_true = []
    y_pred = []

    for _, row in df.iterrows():
        query_id = int(row["query_id"])
        gt = str(gt_dict.get(query_id, "")).strip()

        try:
            pred_ids = eval(row["top_5_case_ids"])
            if not isinstance(pred_ids, list):
                pred_ids = []
            pred_ids = [str(i).strip() for i in pred_ids]
        except Exception:
            pred_ids = []

        hit = gt in pred_ids[:k]
        y_true.append(1)
        y_pred.append(1 if hit else 0)

    return {
        "model": model_name,
        "accuracy": accuracy_score(y_true, y_pred),
        "precision": precision_score(y_true, y_pred, zero_division=0),
        "recall": recall_score(y_true, y_pred, zero_division=0),
        "f1_score": f1_score(y_true, y_pred, zero_division=0)
    }

# === EVALUASI SVM PREDICTION (TOP-1) ===
def eval_prediction(pred_file: str, model_name: str) -> Dict:
    df = pd.read_csv(pred_file)
    y_true, y_pred = [], []

    for _, row in df.iterrows():
        query_id = int(row["query_id"])
        gt = str(gt_dict.get(query_id, "")).strip()
        pred = str(row.get("top_1_case_id", "")).strip()

        if not pred:
            continue

        hit = pred == gt
        y_true.append(1)
        y_pred.append(1 if hit else 0)

    if not y_true:
        print(f"[PERINGATAN] Tidak ada prediksi valid untuk model {model_name}")
        return {
            "model": model_name,
            "accuracy": 0.0,
            "precision": 0.0,
            "recall": 0.0,
            "f1_score": 0.0
        }

    return {
        "model": model_name,
        "accuracy": accuracy_score(y_true, y_pred),
        "precision": precision_score(y_true, y_pred, zero_division=0),
        "recall": recall_score(y_true, y_pred, zero_division=0),
        "f1_score": f1_score(y_true, y_pred, zero_division=0)
    }

# === SIMPAN KASUS GAGAL ===
def save_errors(pred_file: str, output_file: str, top_k: int = 5, mode: str = 'cosine'):
    df = pd.read_csv(pred_file)
    error_cases = []

    for _, row in df.iterrows():
        query_id = int(row["query_id"])
        gt = str(gt_dict.get(query_id, "")).strip()

        if mode == 'cosine':
            try:
                pred_ids = eval(row["top_5_case_ids"])
                if not isinstance(pred_ids, list):
                    pred_ids = []
                pred_ids = [str(i).strip() for i in pred_ids]
            except Exception:
                pred_ids = []
            hit = gt in pred_ids[:top_k]
        else:
            pred_ids = [str(row.get("top_1_case_id", "")).strip()]
            hit = gt == pred_ids[0]

        if not hit:
            error_cases.append({
                "query_id": query_id,
                "query": row["query"],
                "predicted": pred_ids,
                "ground_truth": gt
            })

    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(error_cases, f, indent=2, ensure_ascii=False)

# === EKSEKUSI & SIMPAN HASIL ===
retrieval_metrics = eval_retrieval(COSINE_PRED, "TF-IDF + Cosine")
prediction_metrics = eval_prediction(SVM_PRED, "TF-IDF + SVM")

pd.DataFrame([retrieval_metrics]).to_csv(RETRIEVAL_METRIC_OUTPUT, index=False)
pd.DataFrame([prediction_metrics]).to_csv(PREDICTION_METRIC_OUTPUT, index=False)

save_errors(COSINE_PRED, ERROR_COSINE_OUTPUT, mode='cosine')
save_errors(SVM_PRED, ERROR_SVM_OUTPUT, mode='svm')

# === VISUALISASI ===
combined_df = pd.DataFrame([retrieval_metrics, prediction_metrics])
if not combined_df.empty:
    combined_df.set_index("model")[["accuracy", "precision", "recall", "f1_score"]].plot(
        kind="bar", figsize=(10, 6), title="Perbandingan Model"
    )
    plt.ylabel("Score")
    plt.xticks(rotation=0)
    plt.tight_layout()
    plt.savefig("data/eval/performance_comparison.png")
    plt.close()

print("[SUKSES] Tahap 5 - Evaluasi Model selesai")
print(f"- Hasil evaluasi retrieval disimpan di: {RETRIEVAL_METRIC_OUTPUT}")
print(f"- Hasil evaluasi prediksi disimpan di : {PREDICTION_METRIC_OUTPUT}")
print(f"- Visualisasi disimpan di             : data/eval/performance_comparison.png")
print(f"- Error cosine disimpan di           : {ERROR_COSINE_OUTPUT}")
print(f"- Error svm disimpan di              : {ERROR_SVM_OUTPUT}")


[SUKSES] Tahap 5 - Evaluasi Model selesai
- Hasil evaluasi retrieval disimpan di: data/eval/retrieval_metrics.csv
- Hasil evaluasi prediksi disimpan di : data/eval/prediction_metrics.csv
- Visualisasi disimpan di             : data/eval/performance_comparison.png
- Error cosine disimpan di           : data/eval/error_cases_cosine.json
- Error svm disimpan di              : data/eval/error_cases_svm.json


---